# IMEC2001 Herramientas Computacionales 
## Semana 4: Raíces de Ecuaciones
### Clase 8:  Factor de Fricción de Darcy

Universidad de los Andes — Octubre 30, 2023.

---

## TABLA DE CONTENIDO

### Sección 1: Factor de Fricción de Darcy [→](#section1)
- 1.1. Introducción
- 1.2. Número de Reynolds
- 1.3. Coeficiente de Fricción
- 1.4. Flujo Laminar
- 1.5. Flujo Turbulento
- 1.6. Diagrama de Moody
- 1.7. Ejemplo
    - 1.7.1. Paso 1. Determinar el Número de Reynolds ($Re$)
    - 1.7.2. Paso 2. Determinar la Rugosidad Relativa ($\varepsilon_r$)
    - 1.7.3. Paso 3. Determinar el Coeficiente de Fricción de Darcy ($f$)
    - 1.7.3. Paso 4. Determinar la Pérdida de Carga ($h_f$)
- 1.8. Factor de Fricción de Darcy con Librerías
    - 1.8.1. Empleando `scipy.optimize.fsolve`
    - 1.8.2. Empleando `scipy.optimize.root_scalar`
    - 18.3. Empleando `sympy`
    - 18.4. Empleando `colebrook`
___

<a id="section1"></a>
# Sección 1: Factor de Fricción de Darcy

## 1.1. Introducción

Consideramos la siguiente pregunta:

<div class="alert alert-block alert-success">
Dada una tubería cilíndrica y componentes adicionales como válvulas y codos, más el caudal de diseño (dado por la bomba centrífuga) y las propiedades del fluido, ¿qué diferencia de presiones se necesita para generar el flujo?
</div>

Notemos una relación importante: **diferencia de presión** a causa del **flujo en un conducto**. Algunas variables que están presentes son:
- Rugosidad de la pared.
- Número de Reynolds que indica el tipo de flujo (laminar o turbulento).
- Caudal.

## 1.2. Número de Reynolds

Relación entre las *fuerzas de inercia* y las *fuerzas viscosas* de un fluido que permiten determinar si el flujo es laminar o turbulento.

$$
Re = \frac{\rho V D}{\mu} = \frac{V D}{\nu}
$$

Siendo $\rho$ la densidad del fluido, $V$ la velocidad de flujo, $D$ el diámetro interno de la tubería, $\mu$ la viscosidad dinámica del fluido y $\nu$ la viscosidad cinemática del fluido.

La **viscosidad dinámica $\mu$** es la resistencia interna entre las moléculas de un fluido en movimiento y determina las fuerzas que lo mueven y deforman. Por otra parte, la **viscosidad cinemática $\nu$** se puede entender como la resistencia del fluido al movimiento.

Típicamente:
- Flujo laminar cuando $Re \leq 2300$
- Flujo turbulento cuando $Re > 2300$

Además, el flujo turbulento es más común que el laminar.

<img src='./img/flows.gif' width='350' height='350' />

## 1.3. Coeficiente de Fricción

La fricción dada por las paredes del conducto causan una pérdida de carga $h_f$ (es decir, **reducen la cabeza**).

$$
h_f = f \frac{L}{D} \frac{V^2}{2g}
$$

Siendo:
- $f$ el **coeficiente de fricción de Darcy**.
- $L$ la longitud de la tubería.
- $D$ el diámetro de la tubería.
- $V$ la velocidad del fluido.
- $g$ la gravedad.

En general, el coeficiente de fricción de Darcy es función de $Re$ y la rugosidad de la pared $\varepsilon$.

También, recordemos que:

$$
Q = VA \rightarrow V = \frac{Q}{A}
$$

y

$$
A = \frac{\pi D^2}{4}
$$

Siendo $A$ el área transversal de la tubería cilíndrica.

## 1.4. Flujo Laminar

Recordemos que el flujo dentro de una tubería es *laminar* cuando $Re \leq 2300$. Para este caso, el coeficiente de fricción de Darcy es: <br><br>

<font color="white">
    
$$
f = \frac{64}{Re}
$$
    
</font>

**Nota:** La derivación de esta relación se presenta en la Sección 6.4 de la lectura White - Mecánica de Fluidos (Ch. 6) en Bloque Neón > Contenido > Semana 4 > Lecturas > White, Mecánica de Fluidos (Ch. 6).

## 1.5. Flujo Turbulento

Recordemos que el flujo dentro de una tubería es *turbulento* cuando $Re > 2300$. Para este caso, el coeficiente de fricción de Darcy es: <br><br>

<font color="white">

$$
\frac{1}{\sqrt{f}} = -2 \: \text{log} \left( \frac{\varepsilon / D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right)
$$

</font>

Esta ecuación es también conocida como la **ecuación de Colebrook-White**.

**Nota:** La derivación de esta relación se presenta en la Sección 6.6 de la lectura White - Mecánica de Fluidos (Ch. 6) en Bloque Neón > Contenido > Semana 4 > Lecturas > White, Mecánica de Fluidos (Ch. 6).

## 1.6. Diagrama de Moody

Aquí surge el concepto de *rugosidad relativa* ( $\varepsilon_r$ ), que es la relación entre la rugosidad y el diámetro ( $\varepsilon / D$ ).

<img src='./img/moody2.png' width='800' height='800' />

## 1.7. Ejemplo

Primero, asegurémonos de haber instalado las librerías:

> ```python
  !pip install Pint
  ```

In [ ]:
# Datos y Gráficas
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

# Raíces
import numpy as np
from scipy import optimize
import sympy

# Unidades
from pint import UnitRegistry

## 1.7. Ejemplo

Calculemos la pérdida de carga $h_f$ en una tubería horizontal de 6 in de diámetro ( $D$ ) y 200 ft de longitud ( $L$ ) de hierro fundido asfáltico, por el que circula agua a una velocidad media de 6 ft/s ( $V$ ).

Consideremos:
- $\rho = 1.94$ slug / ft$^3$
- $\mu = 2.09 · 10^{-5}$ slug / ft·s
- $\varepsilon = 0.0004$ ft.

Recordemos que la rugosidad relativa es $\varepsilon_r = \varepsilon / D$.

Podemos utilizar `Pint()` para manejar las unidades de las variables:

<div class='alert alert-block alert-info'>   
    
<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información en la documentación oficial de la librería `pint` dando clic [aquí](https://pint.readthedocs.io/en/stable/getting/tutorial.html).
</div>

In [ ]:
# PASO 1. Definimos las variables
units = UnitRegistry() # Unidades con Pint

D = 6 * units.inch # Diámetro [in]
L = 200 * units.foot # Longitud [ft]
V = 6.0 * units.foot_per_second # Velocidad [ft/s]
rho = 1.94 * units.slug / units.foot**3 # Densidad [slug/ft3]
vis = 2.09e-5 * units.slug / (units.foot * units.second) # Viscosidad [slug/ft·s]
rug = 0.0004 * units.foot # Rugosidad [ft]

O también se puede de la forma:

In [ ]:
D = units.Quantity('6 in') # Diámetro [in]
L = units.Quantity('200 ft') # Longitud [ft]
V = units.Quantity('6 ft/s') # Velocidad [ft/s]
rho = units.Quantity('1.94 slug/ft**3') # Densidad [slug/ft3]
vis = units.Quantity('2.09e-5 slug/(ft*s)') # Viscosidad [slug/ft·s]
rug = units.Quantity('0.0004 ft') # Rugosidad [ft]

In [ ]:
# Variable
print(rho)

# Magnitud
print(rho.magnitude)

# Unidades
print(rho.units)

# Dimensiones
print(rho.dimensionality)

In [ ]:
# Convesión de unidades
print( D.to('foot') ) # Método 1
print( D.to(units.foot) ) # Método 2

In [ ]:
print( D.to('meter') ) # Diámetro [m]
print( L.to('meter') ) # Longitud [m]
print( V.to('meter/second') ) # Velocidad [m/s]
print( rho.to(units.kilogram / units.meter**3) ) # Densidad [kg/m3]
print( vis.to(units.kilogram / (units.meter * units.second)) ) # Viscosidad [kg/m·s]
print( rug.to('meter') ) # Rugosidad [m]

In [ ]:
print( vis.to(units.kilogram / (units.meter * units.second)).to_compact() ) # Viscosidad
print( rug.to('meter').to_compact() ) # Rugosidad

También podemos utilizar la función `to_base_units()` para convertir todas las variables al Sistema Internacional.

<div class='alert alert-block alert-info'>   
    
<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información de las unidades definidas en el repositorio oficial de la librería `pint` dando clic [aquí](https://github.com/hgrecco/pint/blob/master/pint/default_en.txt).
</div>

In [ ]:
print( D.to_base_units() ) # Diámetro [m]
print( L.to_base_units() ) # Longitud [m]
print( V.to_base_units() ) # Velocidad [m/s]
print( rho.to_base_units() ) # Densidad [kg/m3]
print( vis.to_base_units() ) # Viscosidad [kg/m·s]
print( rug.to_base_units() ) # Rugosidad [m]

O podemos calcular las conversiones en forma convencional.

In [ ]:
# PASO 1. Definimos las variables

D = 6/12 # Diámetro [ft]
L = 200 # Longitud [ft]
V = 6.0 # Velocidad [ft/s]
rho = 1.94 # Densidad [slug/ft3]
vis = 2.09e-5 # Viscosidad [slug/ft·s]
rug = 0.0004 # Rugosidad [ft]

### 1.7.1. Paso 1. Determinar el Número de Reynolds ($Re$)

$$
Re = \frac{\rho V D}{\mu}
$$

In [ ]:
def reynolds(densidad, velocidad, diametro, viscosidad):
    return densidad*velocidad*diametro / viscosidad   

Re = reynolds(densidad=rho,
              velocidad=V,
              diametro=D,
              viscosidad=vis)
    
Re

### 1.7.2. Paso 2. Determinar la Rugosidad Relativa ($\varepsilon_{r}$)

Recordemos que:

$$
\varepsilon_r = \varepsilon / D
$$

In [ ]:
rug_rel = rug / D
rug_rel

### 1.7.3. Paso 3. Determinar el Coeficiente de Fricción de Darcy ( $f$ )

<div class="alert alert-block alert-success">
    
Este paso puede realizarse de dos formas:
1. Analíticamente mediante iteraciones hasta alcanzar una convergencia.
2. Diagrama de Moody.

</div>

#### Forma 1
Para el primer método, asumimos un valor de $f$ inicial para luego comparar las dos partes de la igualdad. El valor de $f$ correcto será el que converja según el valor estipulado manualmente de umbral. Reordenando un poco:

$$
\frac{1}{\sqrt{f}} = -2 \: \text{log} \left( \frac{\varepsilon / D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right)
$$

$$
\left( \frac{1}{\sqrt{f}} \right) ^2 = \left[ -2 \: \text{log} \left( \frac{\varepsilon / D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right) \right]^2
$$

$$
\frac{1}{f} = \left[ -2 \: \text{log} \left( \frac{\varepsilon / D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right) \right]^2
$$

$$
f = \frac{1}{\left[ -2 \: \text{log} \left( \frac{\varepsilon / D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right) \right]^2}
$$

#### Forma 2
Con la segunda opción, el coeficiente de fricción de Darcy (en este caso es aproximadamente 0.02) se determina así:

<img src='./img/moody_e1.png' width='800' height='800' />

Vamos a desarrollar el método iterativo para solucionar la ecuación de Colebrook-White:

$$
\frac{1}{\sqrt{f}} = -2 \: \text{log} \left( \frac{\varepsilon / D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right)
$$

Reorganizada:

$$
f = \frac{1}{\left[ -2 \: \text{log} \left( \frac{\varepsilon / D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right) \right]^2}
$$

$$
f = \frac{1}{\left[ -2 \: \text{log} \left( \frac{\varepsilon_r}{3.7} + \frac{2.51}{Re \sqrt{f}} \right) \right]^2}
$$

In [ ]:
# PASO 1. Definimos el lado derecho de la igualdad
def lado_derecho(rug_relativa, reynolds, coef_friccion):
    
    valor = 1 / ( -2 * np.log10( (rug_relativa/3.7) + (2.51 / (reynolds*np.sqrt(coef_friccion))) ) )**2
    
    return valor

In [ ]:
# PASO 3. Iteración con while loop

## 3.1. Definimos nuestros criterio de convergencia
criterio_convergencia = 0.005

In [ ]:
## 3.2. Primera estimación manual
f_asumido = 0.1

valor_lado_derecho = lado_derecho(rug_relativa=rug_rel,
                                  reynolds=Re,
                                  coef_friccion=f_asumido)

print(valor_lado_derecho)

Entonces:

1. Asumimos un valor inicial de $f$. En este primer caso: $f=0.1$.
2. Evaluamos el lado derecho de la ecuación reorganizada. En este primer caso el resultado es $0.019$.

¿Qué criterio utilizamos para saber si este resultado es aceptable?

In [ ]:
np.abs( f_asumido - valor_lado_derecho )

¿Esta diferencia ($0.08$) es menor o igual a nuestro criterio de convergencia ($0.005$)?

In [ ]:
## 3.3. Segunda estimación manual
f_asumido = 0.019

valor_lado_derecho = lado_derecho(rug_relativa=rug_rel,
                                  reynolds=Re,
                                  coef_friccion=f_asumido)

print(valor_lado_derecho)

print( np.abs( f_asumido - valor_lado_derecho ) )

Esta vez la diferencia ($0.0008$) **sí** es menor a nuestro criterio de convergencia ($0.005$), entonces podemos decir que este resultado **es aceptable**.

<div class="alert alert-block alert-success">
    
Notemos que con el método manual obtenemos que $f = 0.0198$ mientras que con el método gráfico (Diagrama de Moody) el resultado es $f = 0.02$.

</div>

In [ ]:
## 3.4. Creamos el while loop
dif = 100

f_asumido = 0.1

i = 1
while dif > criterio_convergencia:
    # Evaluar lado derecho asumiendo un valor f inicial
    valor_lado_derecho = lado_derecho(rug_relativa=rug_rel,
                                      reynolds=Re,
                                      coef_friccion=f_asumido)
    
    # Calcular la diferencia entre el valor asumido y el calculado
    dif = np.abs(f_asumido - valor_lado_derecho)
    
    print(f'ITERACIÓN {i} | Asumido: {f_asumido} | Estimado: {valor_lado_derecho} | Dif: {dif}')
    
    # Nuevo valor de f asumido
    f_asumido = valor_lado_derecho
    i += 1 # i = i + 1

In [ ]:
print(f_asumido)

### 1.7.3. Paso 4. Determinar la Pérdida de Carga ($h_f$)

Recordemos la relación:

$$
h_f = f \frac{L}{D} \frac{V^2}{2g}
$$

En este caso $g = 32.2$ ft/s$^2$.

In [ ]:
# PASO 1. Creamos la función
def perdida_carga(coef_friccion, longitud, diametro, velocidad):
    
    hf = coef_friccion * (longitud/diametro) * ((velocidad**2)/(2*32.2))
   
    return hf

In [ ]:
# PASO 2. Evaluamos la función
hf = perdida_carga(coef_friccion=f_asumido, 
                   longitud=L, 
                   diametro=D, 
                   velocidad=V)

hf

Este resultado quiere decir que la altura manométrica que lograba alcanzar la bomba centrífuga en condiciones iniciales se ve reducida en $4.43$ ft.

<div class="alert alert-block alert-success">
    
Por ejemplo, si inicialmente con la bomba centrífuga se lograba una altura manométrica de $10$ ft, ahora la cabeza será de $10 - 4.43$ ft, es decir, $5.57$ ft. Esto a causa de la fricción dentro de la tubería.

</div>

## 1.8. Factor de Fricción de Darcy con Librerías

Partimos con la ecuación de Colebrook-White:

$$
\frac{1}{\sqrt{f}} = -2 \: \text{log} \left( \frac{\varepsilon / D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right)
$$

Reorganizada para igualarla a cero:

$$
\frac{1}{\sqrt{f}} + 2 \: \text{log} \left( \frac{\varepsilon / D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right) = 0
$$

$$
\frac{1}{\sqrt{f}} + 2 \: \text{log} \left( \frac{\varepsilon_r}{3.7} + \frac{2.51}{Re \sqrt{f}} \right) = 0
$$

### 1.8.1. Empleando `scipy.optimize.fsolve`

<div class='alert alert-block alert-info'>   
    
<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información en la documentación oficial de la librería `scipy.optimize.fsolve` dando clic [aquí](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html).
</div>

In [ ]:
# PASO 1. Definir la función
def colebrook_white(coef_friccion, rug_relativa, reynolds):
    
    valor = (1 / np.sqrt(coef_friccion)) + (2 * np.log10( (rug_relativa/3.7) + (2.51 / (reynolds*np.sqrt(coef_friccion))) ) )
    
    return valor

In [ ]:
# PASO 2. Indicar valor cercano a la raíz
f_asumido = 0.02

In [ ]:
# PASO 3. Estimar raíz
'''
Note que la el parámetro 'args=()' de la función optimize.fsolve()
se agrega porque nuestra función de estudio 'colebrook_white()'
tiene más de un parámetro de entrada
'''
raices = optimize.fsolve(func=colebrook_white, # Como entrada requiere la función 'colebrook_white'
                         x0=f_asumido,
                         args=(rug_rel, Re)) # Note el parámetro args=()


raices

### 1.8.2. Empleando `scipy.optimize.root_scalar`

<div class='alert alert-block alert-info'>   
    
<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información en la documentación oficial de la librería `scipy.optimize.root_scalar` dando clic [aquí](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html).
</div>

In [ ]:
# PASO 1. Definir la función
def colebrook_white(coef_friccion, rug_relativa, reynolds):
    
    valor = (1 / np.sqrt(coef_friccion)) + (2 * np.log10( (rug_relativa/3.7) + (2.51 / (reynolds*np.sqrt(coef_friccion))) ) )
    
    return valor

In [ ]:
# PASO 2. Indicar el rango donde se encuentra la raíz
rango = [0.008, 0.1]

In [ ]:
# PASO 3. Estimar raíces
raices = optimize.root_scalar(f=colebrook_white,
                              bracket=rango,
                              method='bisect',
                              args=(rug_rel, Re))

raices

### 18.3. Empleando `sympy`

<div class='alert alert-block alert-info'>   
    
<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información en la documentación oficial de la librería `sympy.solvers.solveset.nonlinsolve` dando clic [aquí](https://docs.sympy.org/latest/modules/solvers/solveset.html#nonlinsolve).
</div>

In [ ]:
# PASO 1. Definir la variable simbólica
coef_friccion = sympy.Symbol('f', real=True)
coef_friccion

In [ ]:
# PASO 2. Definir la función simbólica
colebrook_white = (1 / coef_friccion**0.5) + (2 * sympy.log( (rug_rel/3.7) + (2.51 / (Re*coef_friccion**0.5) ) ) ) 
colebrook_white

In [ ]:
# PASO 3. Estimar raíces
raices = sympy.nonlinsolve([colebrook_white], [coef_friccion]) # Los parámetros son `([ecuaciones], [símbolos])`; cada uno es una lista
raices

<div class="alert alert-block alert-warning">   

**BONO**
    
Implemente la función `sympy.nonlinsolve` para este caso. Explore cómo obtener un resultado numérico ($0.0198$ en este caso) a partir del modelamiento simbólico.

</div>

### 18.4. Empleando `colebrook`

> ```python
  pip install colebrook
  ```

<div class='alert alert-block alert-info'>   
    
<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información en la documentación oficial de la librería `colebrook.sjFriction` dando clic [aquí](https://github.com/IMEConsultants/colebrook/).
</div>

In [ ]:
!pip install colebrook

In [ ]:
import colebrook

In [ ]:
coef_friccion = colebrook.sjFriction(reynolds=Re,
                                     roughness=rug_rel)

In [ ]:
coef_friccion